# Data Portfolio: Predicting U.S. Interest Rate Change
    Xander de la Bruere
    Jakob Gertsch
    Sam Layton
    Matt Mella
    Wilson Stoddard

In [2]:
import numpy as np

### Research Question Overview

One of the most important and influential indicators of the United States economy is the federal funds rate, commonly referred to as the interest rate. The federal funds rate is a target interest rate set by the Federal Reserve in an attempt to control other key economic indicators. The Federal Reserve seeks to reach its target interest rate by setting a discount rate (the cost banks face to borrow money from other banks in order to meet overnight reserve requirements), engaging in open market operations (reducing or increasing the quantity of U.S. treasury bills available for purchase), and adjusting reserve requirement (the percentage of money banks must keep in reserve rather than lending it to clients) (#1). In practice, the federal funds rate really does prevail as the standard interest rate throughout the economy.

Significantly, an increase or decrease in the federal funds rate impacts levels of personal consumption and spending as well as corporate investment (higher interest rates increase cost to companies of borrowing money). For example, when interest rates are relaively high, consumers have higher incentive to save money (and get interest) rather than spending, and companies have lower incentive to invest (since they must borrow money at a high interest rate to do so), leading to the economy cooling off as a whole. Conversely, lower interest rates increase personal consumption and corporate investment, leading to the economy heating up as a whole. Cumulatively, these personal and corporate decisions, all based on the federal funds rate, have direct and dramatic effects on important economic indicators such as inflation, mortgage prices, unemployment, and even the stock market.

Since the federal funds rate influences so many economic outcomes, foreknowledge of changes in the federal funds rate would allow strategic economic decision-making and investment that would both maximize gains and minimize or eliminate potential losses. For this reason, much effort has been exerted to predict changes in the target federal funds rate announced periodically by the Federal Open Market Committee (FOMC), the policy-making team of the Federal Reserve (#2). The FOMC has shared some indicators that it considers in its decisions, including various price indices and labor market indicators (#3), but it does not explicitly share its goals or the way it considers or weights these metrics. Thus we seek to use logistic regession and classification to develop a robust model for predicting changes in the target federal funds rate based on available economic indicators.


Citations:
1. https://www.federalreserve.gov/monetarypolicy/fomc.htm
2. https://www.investopedia.com/terms/f/federalfundsrate.asp
3. https://federalreserve.gov/monetarypolicy/monetary-policy-what-are-its-goals-how-does-it-work.htm